In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as M
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.models import  resnet50, ResNet50_Weights

In [2]:
# Transformation on Train datasets
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

# Transformation on Test Datasets
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [3]:
# Loading the both train and test data sets from torchvision.datasets
train_data = datasets.CIFAR10(root='./data', transform=transform_train, download=False, train=True)
test_data = datasets.CIFAR10(root='./data', transform=transform_test, download=False, train=False)

In [4]:
# Data loaders to load data in batches from memory for efficient Memory handling and faster training
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [18]:
# Load the pretrain model
model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [19]:
for params in model.parameters():
    params.require_grad = False


# Replace the fully connected layers
num_features = model.fc.in_features # Retrieves the number of features or neuron that is connected to fully connected Layer
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(256, 10)
)


# Moving to GPU if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [21]:
# Function to train the model

def TrainModel(model, train, criterion, optimizer, epochs=10):
    model.train() # Set the model mode to Train i.e Activate batch normalization and dropout layers
    running_loss = 0.0
    correct = 0
    total = 0
    for i in range(epochs):
        for input, label in train:
            input, label = input.to(device), label.to(device)
            
            # Forwad pass
            y_pred = model(input)
            loss = criterion(y_pred, label)
            
            # Backprobagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = y_pred.max(1)
            total += label.size(0)
            correct += predicted.eq(label).sum().item()
        return running_loss / len(train_loader), 100. * correct / total

In [ ]:
TrainModel(model=model, train=train_loader, criterion=criterion, optimizer=optimizer)